In [1]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15191 sha256=92f65fae4047419c987bfcecdda8d0427a2ea58ec530f022c83b4c87c6df2272
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [2]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt

import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime

In [3]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [4]:
# calling API 
data=YahooFinancials('ETH-USD').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
eth_df = pd.DataFrame(data['ETH-USD']['prices'])

eth_df =eth_df.drop(['high','low','open','volume','date','adjclose'], axis=1)
eth_df.rename(columns={'formatted_date':'date'},inplace=True)
eth_df['date']=pd.to_datetime(eth_df['date'])

In [5]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [6]:
# Scale data for ML engine
scaler = MinMaxScaler()
eth_df['scaled_close'] = scaler.fit_transform(np.expand_dims(eth_df['close'].values, axis=1))

In [7]:
def PrepareData(days):
    df = eth_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [8]:
PrepareData(3) # 3 days


(            close       date  scaled_close    future
 0      187.976547 2019-11-07      0.016457  0.016776
 1      184.211472 2019-11-08      0.015656  0.015928
 2      185.028717 2019-11-09      0.015830  0.016216
 3      189.477280 2019-11-10      0.016776  0.016517
 4      185.489639 2019-11-11      0.015928  0.016036
 ...           ...        ...           ...       ...
 1093  1645.093384 2022-11-04      0.326384  0.310112
 1094  1627.968018 2022-11-05      0.322741  0.259967
 1095  1572.234741 2022-11-06      0.310887  0.210479
 1096  1568.591309 2022-11-07      0.310112  0.252869
 1097  1332.835571 2022-11-08      0.259967  0.240382
 
 [1098 rows x 4 columns], array([[0.29971528],
        [0.30223152],
        [0.32638383],
        [0.32274127],
        [0.31088686],
        [0.3101119 ],
        [0.25996694],
        [0.21047917],
        [0.25286895],
        [0.24038167]], dtype=float32), array([[[0.01645665810918889, Timestamp('2019-11-07 00:00:00')],
         [0.01565583061

In [9]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [10]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
137/137 [==============================] - 5s 9ms/step - loss: 0.0088
Epoch 2/80
137/137 [==============================] - 2s 12ms/step - loss: 0.0034
Epoch 3/80
137/137 [==============================] - 2s 17ms/step - loss: 0.0030
Epoch 4/80
137/137 [==============================] - 2s 15ms/step - loss: 0.0025
Epoch 5/80
137/137 [==============================] - 2s 14ms/step - loss: 0.0024
Epoch 6/80
137/137 [==============================] - 2s 14ms/step - loss: 0.0026
Epoch 7/80
137/137 [==============================] - 2s 16ms/step - loss: 0.0024
Epoch 8/80
137/137 [==============================] - 2s 16ms/step - loss: 0.0019
Epoch 9/80
137/137 [==============================] - 2s 15ms/step - loss: 0.0021
Epoch 10/80
137/137 [==============================] - 2s 15ms/step - loss: 0.0020
Epoch 11/80
137/137 [==============================] - 1s 9ms/step - loss: 0.0019
Epoch 12/80
137/137 [==============================] - 1s 9ms/step - loss: 0.0018
Epoch 13/80
137/

In [11]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (1286.85$, 1273.3$, 1321.47$)


In [12]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
eth_lstm=pd.DataFrame()
eth_lstm['future days']=days
eth_lstm['price']=predictions
eth_lstm['ticker']='ETH'

eth_lstm.to_csv('eth_lstm.csv') 
files.download('eth_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# # Execute model for the whole history range
# copy_df = eth_df.copy()
# y_predicted = model.predict(x_train)
# y_predicted_transformed = np.squeeze(scaler.inverse_transform(y_predicted))
# first_seq = scaler.inverse_transform(np.expand_dims(y_train[:6], axis=1))
# last_seq = scaler.inverse_transform(np.expand_dims(y_train[-3:], axis=1))
# y_predicted_transformed = np.append(first_seq, y_predicted_transformed)
# y_predicted_transformed = np.append(y_predicted_transformed, last_seq)
# copy_df[f'predicted_close'] = y_predicted_transformed

# # Add predicted results to the table
# date_now = dt.date.today()
# date_tomorrow = dt.date.today() + dt.timedelta(days=1)
# date_after_tomorrow = dt.date.today() + dt.timedelta(days=2)

# copy_df.loc[date_now] = [predictions[0], f'{date_now}', 0, 0]
# copy_df.loc[date_tomorrow] = [predictions[1], f'{date_tomorrow}', 0, 0]
# copy_df.loc[date_after_tomorrow] = [predictions[2], f'{date_after_tomorrow}', 0, 0]

# # Result chart
# plt.style.use(style='ggplot')
# plt.figure(figsize=(16,10))
# plt.plot(copy_df['close'][-150:].head(147))
# plt.plot(copy_df['predicted_close'][-150:].head(147), linewidth=1, linestyle='dashed')
# # plt.plot(copy_df['close'][-150:].tail(4))
# plt.xlabel('days')
# plt.ylabel('price')
# plt.legend([f'Actual price for {crypto}', 
#             f'Predicted price for {crypto}',
#             f'Predicted price for future 3 days'])
# plt.show()